In [63]:
import pandas as pd

In [64]:
df1 = pd.read_csv('2-fft-normal-n-0-3-data.csv')
df2 = pd.read_csv('2-fft-normal-n-0-5-data.csv')
df3 = pd.read_csv('2-fft-normal-n-0-10-data.csv')
df4 = pd.read_csv('2-fft-normal-n-0-12-data.csv')
df5 = pd.read_csv('2-fft-normal-n-0-15-data.csv')

In [65]:
df1 = df1.loc[df1['router'] == 1]
df2 = df2.loc[df2['router'] == 1]
df3 = df3.loc[df3['router'] == 1]
df4 = df4.loc[df4['router'] == 1]
df5 = df5.loc[df5['router'] == 1]

In [66]:
df1 = df1.drop(columns=['router'])
df2 = df2.drop(columns=['router'])
df3 = df3.drop(columns=['router'])
df4 = df4.drop(columns=['router'])
df5 = df5.drop(columns=['router'])

In [67]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-good.csv',index=False)
df2.to_csv('2-fft-normal-n-0-5-data-r1-good.csv',index=False)
df3.to_csv('2-fft-normal-n-0-10-data-r1-good.csv',index=False)
df4.to_csv('2-fft-normal-n-0-12-data-r1-good.csv',index=False)
df5.to_csv('2-fft-normal-n-0-15-data-r1-good.csv',index=False)

In [68]:
df1 = pd.read_csv('2-fft-normal-n-0-3-data-r1-good.csv')
df2 = pd.read_csv('2-fft-normal-n-0-5-data-r1-good.csv')
df3 = pd.read_csv('2-fft-normal-n-0-10-data-r1-good.csv')
df4 = pd.read_csv('2-fft-normal-n-0-12-data-r1-good.csv')
df5 = pd.read_csv('2-fft-normal-n-0-15-data-r1-good.csv')

In [69]:
def timecount(df):
    timearr = []
    interval = 99
    count = 0
    for index, row in df.iterrows():
        if row["timestamp"]<=interval:
            count+=1
        else:
            timearr.append([interval+1,count])
            count=1
            interval+=100
    timearr.append([interval+1,count])
    return timearr

In [70]:
def maxcount(timearr,df):
    countarr = []
    increarr = []
    maxarr = []
    for i in range(len(timearr)):
        for cnt in range(timearr[i][1],0,-1):
            countarr.append(cnt)
            maxarr.append(timearr[i][1])
            increment = timearr[i][1] - cnt + 1
            increarr.append(increment)
    df = df.assign(packet_count_decr=countarr)
    df = df.assign(packet_count_incr=increarr)
    df = df.assign(max_packet_count=maxarr)
    return df

In [71]:
df1 = maxcount(timecount(df1),df1)
df2 = maxcount(timecount(df2),df2)
df3 = maxcount(timecount(df3),df3)
df4 = maxcount(timecount(df4),df4)
df5 = maxcount(timecount(df5),df5)

In [72]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,dst_ni,dst_router,traversal_id,hop_count,current_hop,hop_percentage,enqueue_time,packet_count_decr,packet_count_incr,max_packet_count
0,11,2,4,1,1,4,3,9,0,0,23,7,0,4,1,0.250000,8,2,1,2
1,84,4,2,1,1,4,3,9,3,3,24,8,2,5,2,0.400000,12,1,2,2
2,216,2,4,1,1,4,3,9,0,0,22,6,5,3,1,0.333333,8,1,1,1
3,342,2,4,1,1,4,3,9,0,0,23,7,7,4,1,0.250000,8,1,1,1
4,388,1,2,1,1,4,3,9,3,3,21,5,8,3,2,0.666667,12,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56912,3268816,4,2,6,2,2,3,9,18,2,20,4,20041,3,1,0.333333,15,5,1,5
56913,3268823,4,2,6,3,2,3,9,18,2,20,4,20041,3,1,0.333333,22,4,2,5
56914,3268830,4,2,6,4,2,3,9,18,2,20,4,20041,3,1,0.333333,29,3,3,5
56915,3268837,4,2,6,5,3,3,9,18,2,20,4,20041,3,1,0.333333,36,2,4,5


In [73]:
def rename(df):
    df["packet_count_index"] = df["packet_count_decr"]*df["packet_count_incr"]
    df["packet_max_index"] = df["packet_count_index"]*df["max_packet_count"]
    df["port_index"] = df["outport"]*df["inport"]
    df["cache_coherence_flit_index"] = df["cache_coherence_type"]*df["flit_id"]
    df["flit_index"] = df["cache_coherence_flit_index"]*df["flit_type"]
    df["traversal_index"] = df["flit_index"]*df["traversal_id"]
    df["cache_coherence_vnet_index"] = df["cache_coherence_type"]*df["vnet"]
    df["vnet_vc_index"] = df["vnet"]*df["vc"]
    df["vnet_vc_cc_index"] = df["vnet"]*df["cache_coherence_vnet_index"]
    df.head(50)

In [74]:
rename(df1)
rename(df2)
rename(df3)
rename(df4)
rename(df5)

In [75]:
df1['target'] = 1
df2['target'] = 1
df3['target'] = 1
df4['target'] = 1
df5['target'] = 1

In [76]:
df1

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,2,4,8,1,4,0,3,27,9,1
1,84,4,2,1,1,4,3,9,3,3,...,2,4,8,1,4,8,3,27,9,1
2,216,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,20,3,27,9,1
3,342,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,28,3,27,9,1
4,388,1,2,1,1,4,3,9,3,3,...,1,1,2,1,4,32,3,27,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56912,3268816,4,2,6,2,2,3,9,18,2,...,5,25,8,12,24,480984,18,27,54,1
56913,3268823,4,2,6,3,2,3,9,18,2,...,8,40,8,18,36,721476,18,27,54,1
56914,3268830,4,2,6,4,2,3,9,18,2,...,9,45,8,24,48,961968,18,27,54,1
56915,3268837,4,2,6,5,3,3,9,18,2,...,8,40,8,30,90,1803690,18,27,54,1


In [77]:
df2

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,0,3,27,9,1
1,218,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,20,3,27,9,1
2,336,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,28,3,27,9,1
3,434,1,4,1,1,4,3,9,0,0,...,1,1,4,1,4,36,3,27,9,1
4,470,0,1,1,1,4,3,9,5,5,...,2,4,0,1,4,40,3,27,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46622,3265536,2,4,5,2,2,3,9,0,0,...,5,25,8,10,20,401120,15,27,45,1
46623,3265543,2,4,5,3,2,3,9,0,0,...,8,40,8,15,30,601680,15,27,45,1
46624,3265550,2,4,5,4,2,3,9,0,0,...,9,45,8,20,40,802240,15,27,45,1
46625,3265557,2,4,5,5,3,3,9,0,0,...,8,40,8,25,75,1504200,15,27,45,1


In [78]:
df3

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,0,3,27,9,1
1,218,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,20,3,27,9,1
2,336,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,28,3,27,9,1
3,434,1,4,1,1,4,3,9,0,0,...,1,1,4,1,4,36,3,27,9,1
4,484,0,1,1,1,4,3,9,10,10,...,2,4,0,1,4,40,3,27,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45786,3265495,2,4,5,2,2,3,9,0,0,...,1,1,8,10,20,401180,15,27,45,1
45787,3265502,2,4,5,3,2,3,9,0,0,...,4,16,8,15,30,601770,15,27,45,1
45788,3265509,2,4,5,4,2,3,9,0,0,...,6,24,8,20,40,802360,15,27,45,1
45789,3265516,2,4,5,5,3,3,9,0,0,...,6,24,8,25,75,1504425,15,27,45,1


In [79]:
df4

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,0,3,27,9,1
1,206,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,16,3,27,9,1
2,325,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,24,3,27,9,1
3,424,1,4,1,1,4,3,9,0,0,...,1,1,4,1,4,32,3,27,9,1
4,532,0,4,1,1,4,3,9,0,0,...,1,1,0,1,4,40,3,27,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46083,3272305,4,2,6,3,2,3,9,18,2,...,1,1,8,18,36,721656,18,27,54,1
46084,3272312,4,2,6,4,2,3,9,18,2,...,1,1,8,24,48,962208,18,27,54,1
46085,3272319,4,2,6,5,3,3,9,18,2,...,1,1,8,30,90,1804140,18,27,54,1
46086,3272320,4,2,2,1,4,3,10,18,2,...,1,1,8,2,8,160376,6,30,18,1


In [80]:
df5

,timestamp,outport,inport,cache_coherence_type,flit_id,flit_type,vnet,vc,src_ni,src_router,...,packet_count_index,packet_max_index,port_index,cache_coherence_flit_index,flit_index,traversal_index,cache_coherence_vnet_index,vnet_vc_index,vnet_vc_cc_index,target
0,11,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,0,3,27,9,1
1,220,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,20,3,27,9,1
2,346,2,4,1,1,4,3,9,0,0,...,1,1,8,1,4,28,3,27,9,1
3,444,1,4,1,1,4,3,9,0,0,...,1,1,4,1,4,36,3,27,9,1
4,524,0,1,1,1,4,3,9,15,15,...,1,1,0,1,4,40,3,27,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45526,3275300,4,2,6,2,2,3,9,18,2,...,1,1,8,12,24,481344,18,27,54,1
45527,3275307,4,2,6,3,2,3,9,18,2,...,1,1,8,18,36,722016,18,27,54,1
45528,3275314,4,2,6,4,2,3,9,18,2,...,1,1,8,24,48,962688,18,27,54,1
45529,3275321,4,2,6,5,3,3,9,18,2,...,1,1,8,30,90,1805040,18,27,54,1


In [81]:
df1.dtypes

timestamp                       int64
outport                         int64
inport                          int64
cache_coherence_type            int64
flit_id                         int64
flit_type                       int64
vnet                            int64
vc                              int64
src_ni                          int64
src_router                      int64
dst_ni                          int64
dst_router                      int64
traversal_id                    int64
hop_count                       int64
current_hop                     int64
hop_percentage                float64
enqueue_time                    int64
packet_count_decr               int64
packet_count_incr               int64
max_packet_count                int64
packet_count_index              int64
packet_max_index                int64
port_index                      int64
cache_coherence_flit_index      int64
flit_index                      int64
traversal_index                 int64
cache_cohere

In [82]:
df1.to_csv('2-fft-normal-n-0-3-data-r1-good.csv',index=False)
df2.to_csv('2-fft-normal-n-0-5-data-r1-good.csv',index=False)
df3.to_csv('2-fft-normal-n-0-10-data-r1-good.csv',index=False)
df4.to_csv('2-fft-normal-n-0-12-data-r1-good.csv',index=False)
df5.to_csv('2-fft-normal-n-0-15-data-r1-good.csv',index=False)